In [ ]:
# Header starts here.
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
from copy import deepcopy
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    objc = deepcopy(obj)
    try:
        tmp = DX(str(float(objc)))
        objc = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(objc)):
            tmp = DX(str(float(objc[i])))
            objc[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return objc

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

half = S(1)/2

# Header ends here.
#
# https://colab.research.google.com/github/kassbohm/wb-snippets/blob/master/ipynb/HTM_03/Uebungen/3.1_cc.ipynb

from mpmath import radians, degrees, pi

l1, l2, l3, b = 1*m, 2*m, 2*m, 2*m
p1 = radians(30)
prec = 0.0001


p2, p3 = var("φ₂ φ₃")
dp2, dp3 = var("delta_2, delta_3")

c1, s1 = cos(p1), sin(p1)
c2, s2 = cos(p2), sin(p2)
c3, s3 = cos(p3), sin(p3)

xdev = ( l1*c1 + l2*c2 - l3*c3 - b ) / m
ydev = ( l1*s1 + l2*s2 - l3*s3 ) / m

J11, J12 = diff(xdev, p2), diff(xdev, p3)
J21, J22 = diff(ydev, p2), diff(ydev, p3)

# Jacobian:
J = Matrix([[J11, J12],[J21, J22]])

pprint("\n--- Numerical Solution using SymPy's nsolve ---")

# Initial Values:
pprint("\nInitial (φ₂ φ₃) / deg:")
pprint(Matrix([45, 90]))
p2i, p3i = radians(45), radians(90)

sol = nsolve( [xdev, ydev], [p2, p3], [p2i , p3i], dict=True )
sol = sol[0]
sol =  Matrix([sol[p2], sol[p3]])
pprint("\nnsolve-solution:")
pprint("\n(φ₂ φ₃) / deg:")
tmp = degrees(sol)
pprint(iso_round(tmp,prec))

pprint("\n--- Step-by-Step Newton-Solution ---")
pprint("\n1: Jacobian:")
pprint(J)

# n = 1 Iteration:
pprint("\n2: Inital (φ₂ φ₃) / rad:")
p = Matrix([p2i, p3i])
pprint(iso_round(p, prec))

F = Matrix([xdev, ydev])

sub_list = [
    (p2, p2i),
    (p3, p3i),
    ]

pprint("\n3: Jacobian for (φ₂ φ₃):")
Jn = J.subs(sub_list)
# pprint(Jn)
pprint(iso_round(Jn, prec))

pprint("\n4: Deviation Phi for (φ₂ φ₃) / rad:")
Fn = F.subs(sub_list)
pprint(iso_round(Fn, prec))

# Newton-Step:
pprint("\n5: Increment (δφ₂ δφ₃) / rad:")
dp = Matrix([dp2, dp3])
eq = Eq(Jn*dp, -Fn)
sol = solve(eq,[dp2, dp3], dict=True)
sol = sol[0]
dp = Matrix([sol[dp2], sol[dp3]])
pprint(iso_round(dp, prec))
p += dp

pprint("\n6a: New (φ₂ φ₃) / rad:")
pprint(iso_round(p, prec))

pprint("\n6b: New (φ₂ φ₃) / rad:")
pprint(iso_round(degrees(p), prec))

# --- Numerical Solution using SymPy's nsolve ---
#
# Initial (phi2, phi3) / deg:
# ⎡45⎤
# ⎢  ⎥
# ⎣90⎦
#
# nsolve-solution:
#
# (phi2, phi3) / deg:
# ⎡48.1571⎤
# ⎢       ⎥
# ⎣84.2549⎦
#
# --- Step-by-Step Newton-Solution ---
#
# 1: Jacobian:
# ⎡-2⋅sin(φ₂)  2⋅sin(φ₃) ⎤
# ⎢                      ⎥
# ⎣2⋅cos(φ₂)   -2⋅cos(φ₃)⎦
#
# 2: Init:
# ⎡0.7854⎤
# ⎢      ⎥
# ⎣1.5708⎦
#
# 3: Jacobian for (phi2, phi3):
# ⎡-1.4142  2.0⎤
# ⎢            ⎥
# ⎣1.4142   0.0⎦
#
# 4: Deviation Phi for (phi2, phi3):
# ⎡0.2802 ⎤
# ⎢       ⎥
# ⎣-0.0858⎦
#
# 5: Increment (dphi2, dphi3):
# ⎡0.0607 ⎤
# ⎢       ⎥
# ⎣-0.0972⎦
#
# 6: New (phi2, phi3):
#
# In rad (radians):
# ⎡0.8461⎤
# ⎢      ⎥
# ⎣1.4736⎦
#
# In deg (degrees):
# ⎡48.4756⎤
# ⎢       ⎥
# ⎣84.4293⎦
